# Retention dataset

In [2]:
import pandas as pd
import numpy as np
import pyarrow
from datetime import datetime, date, timedelta
import time

from google.cloud import storage
from google.cloud import bigquery

import sys
import os

sys.path.append(os.path.realpath('../../'))
import data_generator

In [3]:
bigquery_client = bigquery.Client.from_service_account_json('../../credentials/data-analysis-sql-309220-6ce084250abd.json')

In [9]:
params = {
    'dates': {
        'date': {
            'start': '2021-01-01', 
            'end': '2021-03-31'
        }
    }, 
    'categorical': {
        'group': {
            'categories': ['control', 'experimental'],
            'probs': [0.5, 0.5],
            'indices': [1, 1.05]
        },
        'country': {
            'categories': ['UK', 'DE', 'FR', 'IT'], 
            'probs': [0.31, 0.27, 0.23, 0.19], 
            'indices': [0.91, 1.18, 1.05, 0.85]
        }, 
        'platform': {
            'categories': ['android', 'ios', 'web'], 
            'probs': [0.41, 0.29, 0.3], 
            'indices': [0.95, 1.25, 0.8]
        }
    }, 
    'values': {
        'amount': {
            'distribution': 'exponential', 
            'parameters': {
                'scale': 10
            }, 
            'indices': True, 
            'min': 1, 
            'round': 2
        }
    }
}


df = data_generator.create_dataset(rows=10000, params=params, seed=1)
df.head()

,date,group,country,platform,amount
0,2021-01-09,experimental,UK,ios,45.37
1,2021-03-08,control,UK,web,3.14
2,2021-02-18,control,FR,android,32.32
3,2021-03-09,control,FR,android,1.41
4,2021-02-08,experimental,IT,ios,10.49


In [10]:
print(df.sample(frac=0.001).to_markdown(index=False))

| date       | group        | country   | platform   |   amount |
|:-----------|:-------------|:----------|:-----------|---------:|
| 2021-01-22 | control      | DE        | android    |     3.83 |
| 2021-02-13 | control      | IT        | android    |     6.69 |
| 2021-02-01 | experimental | UK        | web        |     3.5  |
| 2021-02-17 | experimental | FR        | android    |    16.96 |
| 2021-03-05 | control      | DE        | web        |    14.15 |
| 2021-02-07 | experimental | IT        | ios        |     1    |
| 2021-01-05 | control      | IT        | web        |    17.44 |
| 2021-02-19 | experimental | UK        | android    |    10.23 |
| 2021-01-24 | control      | DE        | web        |     2.48 |
| 2021-02-05 | experimental | FR        | android    |    27.38 |


### Write the dataset to BigQuery

In [11]:
df.to_csv('payments.csv', index=False)

In [12]:
table_ref = bigquery_client.dataset('synthetic').table('payments')

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1 # ignore the header
job_config.autodetect = True

with open('payments.csv', 'rb') as source_file:
    job = bigquery_client.load_table_from_file(
        source_file, table_ref, job_config=job_config
    )

job.result()

In [13]:
%%bash
rm payments.csv